In [1]:
import tensorflow.keras as keras
import numpy as np

import pathlib
import os

# Carregamento dos dados:

In [2]:
dataset_dir = pathlib.Path(os.getcwd())/"emails"

In [3]:
data = []
for fn in (dataset_dir/"spam").iterdir():
    with open(fn,'r', errors="ignore") as file:
        for line in file:
            if line.startswith("Subject:"):
                subject = line[9:].strip()
                data.append((subject, True))

for fn in (dataset_dir/"ham").iterdir():
    with open(fn,'r', errors="ignore") as file:
        for line in file:
            if line.startswith("Subject:"):
                subject = line[9:].strip()
                data.append((subject, False))    

In [4]:
np.random.shuffle(data)

In [5]:
subjects, is_spam = list(zip(*data))
subjects = np.array(subjects)
is_spam = np.array(is_spam)

## Treinamento começa a partir de aqui:

In [6]:
subjects_train = subjects[:3033]
is_spam_train = is_spam[:3033]
subjects_test = subjects[3033:]
is_spam_test = is_spam[3033:]

In [7]:
camada = keras.layers.TextVectorization(max_tokens=10000, output_mode='binary')
camada.adapt(subjects)

In [8]:
model = keras.Sequential([
        camada,
        keras.layers.Dense(8),
        keras.layers.Dense(8),
        keras.layers.Dense(1)
    ])

In [9]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy'])

In [10]:
history = model.fit(subjects_train,is_spam_train, validation_split=.2, epochs=5)

Epoch 1/5
76/76 [==============================] - 2s 11ms/step - loss: 0.5312 - binary_accuracy: 0.8524 - val_loss: 0.3026 - val_binary_accuracy: 0.8468
Epoch 2/5
76/76 [==============================] - 1s 7ms/step - loss: 0.2176 - binary_accuracy: 0.8792 - val_loss: 0.2949 - val_binary_accuracy: 0.8616
Epoch 3/5
76/76 [==============================] - 1s 7ms/step - loss: 0.1502 - binary_accuracy: 0.9184 - val_loss: 0.2491 - val_binary_accuracy: 0.8863
Epoch 4/5
76/76 [==============================] - 0s 6ms/step - loss: 0.1042 - binary_accuracy: 0.9571 - val_loss: 0.2300 - val_binary_accuracy: 0.8995
Epoch 5/5
76/76 [==============================] - 1s 7ms/step - loss: 0.0721 - binary_accuracy: 0.9815 - val_loss: 0.2221 - val_binary_accuracy: 0.9160


In [11]:
binary_loss, binary_accuracy = model.evaluate(subjects_test,is_spam_test)
print("loss: %s\naccuracy: %s" % (binary_loss, binary_accuracy))

11/11 [==============================] - 0s 3ms/step - loss: 0.1705 - binary_accuracy: 0.9320
loss: 0.17048262059688568
accuracy: 0.9319526553153992


In [12]:
probs = model.predict(subjects_test[0:1])
probs[0,0] > .5, is_spam_test[0]

(False, False)

In [13]:
model.save(dataset_dir.parent)

INFO:tensorflow:Assets written to: C:\Users\mello\OneDrive\Documentos\faculdade\ciencia de dados\notebooks\brincadeiras\spam\assets


In [44]:
i+=1
subjects_test[i], is_spam_test[i]

('[ILUG-Social] re: Guaranteed to lose 10-12 lbs in 30 days 10.148', True)

In [49]:
subjects_test[i]

'[ILUG-Social] re: Guaranteed to lose 10-12 lbs in 30 days 10.148'

In [48]:
probs = model.predict([str(subjects_test[i])])
probs[0,0] > .5, is_spam_test[i]

(True, True)